In [31]:
using POMDPs
using Random # for AbstractRNG
using POMDPModelTools
using Pkg

using DataFrames
Pkg.add("JSON")

POMDPs.add_registry()
using Pkg; Pkg.add("DiscreteValueIteration")

using DiscreteValueIteration
using Pkg
Pkg.add("StaticArrays")
using POMDPSimulators
using POMDPPolicies

using DataFrames
using Statistics

using CSV

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
   Cloning registry from "https://github.com/JuliaPOMDP/Registry"
Fetching: [========================================>]  100.0 %.0 %=========>                               ]  21.4 %]  42.7 %==========================>              ]  64.1 %]  85.3 %]  93.6 %

┌ Info: registry `JuliaPOMDP` already exists in `~/.julia/registries`
└ @ POMDPs /Users/efan/.julia/packages/POMDPs/JiYXY/src/utils.jl:65


 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [43]:

include("./MaximumRewardPath.jl")

setup_agents (generic function with 1 method)

In [3]:
Int64[[[[0] [0]] [[1] [0]]]; [[[0] [0]]  [[0] [1]]]]

2×4 Array{Int64,2}:
 0  0  1  0
 0  0  0  1

In [44]:
using POMDPSimulators
using POMDPPolicies
rng = MersenneTwister(1235);
g_len =2
disc = .95
trans = Dict(1=> [2], 2=>[1])
print(trans)
PMaxRewardPathMDP() = PMaxRewardPathMDP(g_len, .9,
        Float64[[0, .1] [0.9, 0.0]],
        trans)
m = PMaxRewardPathMDP()
@requirements_info ValueIterationSolver() PMaxRewardPathMDP()
# policy that maps every input to an action
policy = FunctionPolicy(s->s==Vec2(1,1) ? 2 : 1)
for (s, a, r) in stepthrough(m, policy, "s,a,r", max_steps=10)
  render(m, (s,a,r))
  println("s,a,r:($s,$a,$r)")
end



Dict(2=>[1],1=>[2])INFO: POMDPs.jl requirements for solve(::ValueIterationSolver, ::Union{MDP,POMDP}) and dependencies. ([✔] = implemented correctly; [X] = missing)

For solve(::ValueIterationSolver, ::Union{MDP,POMDP}):
  [✔] discount(::PMaxRewardPathMDP)
  [✔] n_states(::PMaxRewardPathMDP)
  [✔] n_actions(::PMaxRewardPathMDP)
  [✔] transition(::PMaxRewardPathMDP, ::SArray, ::Int64)
  [✔] reward(::PMaxRewardPathMDP, ::SArray, ::Int64, ::SArray)
  [✔] stateindex(::PMaxRewardPathMDP, ::SArray)
  [✔] actionindex(::PMaxRewardPathMDP, ::Int64)
  [✔] actions(::PMaxRewardPathMDP, ::SArray)
  [✔] support(::Deterministic)
  [✔] pdf(::Deterministic, ::SArray)
For ordered_states(::Union{MDP,POMDP}) (in solve(::ValueIterationSolver, ::Union{MDP,POMDP})):
  [✔] states(::PMaxRewardPathMDP)
For ordered_actions(::Union{MDP,POMDP}) (in solve(::ValueIterationSolver, ::Union{MDP,POMDP})):
  [✔] actions(::PMaxRewardPathMDP)
s,a,r:([1, 1],2,0.9)
s,a,r:([1, 2],1,0.1)
s,a,r:([1, 1],2,0.9)
s,a,r:([1, 2],1,0.

In [5]:
solver = ValueIterationSolver(max_iterations=100, belres=1e-6, include_Q=true) # initializes the Solver type
vip = solve(solver, m)
for (s, a, r) in stepthrough(m, vip, "s,a,r", max_steps=10)
  render(m, (s,a,r))
  println("s,a,r:($s,$a,$r)")
end

s,a,r:([1, 1],2,0.9)
s,a,r:([1, 2],1,0.1)
s,a,r:([1, 1],2,0.9)
s,a,r:([1, 2],1,0.1)
s,a,r:([1, 1],2,0.9)
s,a,r:([1, 2],1,0.1)
s,a,r:([1, 1],2,0.9)
s,a,r:([1, 2],1,0.1)
s,a,r:([1, 1],2,0.9)
s,a,r:([1, 2],1,0.1)


In [42]:
function build_random_path(g_len)
  trans = Dict{Int64, Array{Int64}}()
  for i in 1:g_len
        
        for j in 1:g_len
          # print(rand(Bernoulli(0.2)))
          p = 2 * log(g_len)/g_len
          # print("p:$p")
          if rand(Bernoulli(p)) == 1
                if !(i  in keys(trans))
                    trans[i] = []
                end
                append!(trans[i], j)
          end
        end
  end
  print(trans)

  is = keys(trans)
  return PMaxRewardPathMDP(g_len, .9,
        reshape([ j in trans[i] ? exp(randn(Float64)*4) : 0 for j in 1:g_len for i in is],g_len,g_len),
        trans)
 end

build_random_path (generic function with 1 method)

In [7]:
[]

0-element Array{Any,1}

In [8]:
m = build_random_path(g_len)

p:0.6931471805599453p:0.6931471805599453p:0.6931471805599453p:0.6931471805599453Dict{Int64,Array{Int64,N} where N}(2=>[2],1=>[1, 2])

PMaxRewardPathMDP(2, 0.9, [2.0 4.0; 0.0 6.0], Dict{Int64,Array{Int64,N} where N}(2=>[2],1=>[1, 2]))

In [9]:
print(length(m.trans))

2

In [10]:
print(reshape(m.Rs,1,:))

[2.0 0.0 4.0 6.0]

In [11]:
print(reshape(reshape(m.Rs,1,:),2,2))

[2.0 4.0; 0.0 6.0]

In [12]:
Float64[[0, .1] [0.9, 0.0]]

2×2 Array{Float64,2}:
 0.0  0.9
 0.1  0.0

In [13]:

function ucb_pol(li, policy, priors, i, actions, s)
    best_action = action(policy, s)
    #println("in state: $s, best action:$best_action")
    return action(policy, s)
end


ucb_pol (generic function with 1 method)

In [14]:
curry(f, x) = (xs...) -> f(x, xs...)
function ucb_mdp_builder(rng, true_mdp, priors, i, num_states, num_chains, steps, start_state)
  # build mdp
  new_means = priors.theta + priors.std
  mdp = deepcopy(true_mdp)
  # Should use theta and std
    
  li = LinearIndices(true_mdp.Rs)
  mdp.Rs = reshape([prior.theta + 6* prior.std for prior in eachrow(priors)],num_states,num_states)
  # println("Rs: $(mdp.Rs)")
  # println("prior: $(priors.theta)")              
  solver = ValueIterationSolver(max_iterations=1000, belres=1e-6, include_Q=true)#, verbose=true) # initializes the Solver type
  vip = solve(solver, mdp)
  new_policy = FunctionPolicy(curry(curry(curry(curry(curry(ucb_pol, 1), vip), priors),i), actions))
  if start_state == nothing
    return Iterators.Stateful(stepthrough(deepcopy(true_mdp), new_policy, "s,a,r,sp,t",
                              max_steps=steps))
  else
     return Iterators.Stateful(stepthrough(deepcopy(true_mdp), new_policy, start_state, "s,a,r,sp,t",
                              max_steps=steps))
  end
end

ucb_mdp_builder (generic function with 1 method)

In [41]:
curry(f, x) = (xs...) -> f(x, xs...)
function ssgn_mdp_builder(rng, true_mdp, priors, i, num_states, num_chains, steps, start_state)
  # build mdp
  # new_means = priors.theta + priors.std
  mdp = deepcopy(true_mdp)
    
  li = LinearIndices(true_mdp.Rs)
  mdp.Rs = reshape([randn(rng, Float32) * (2*priors.std[j]) + prior.theta  for prior in eachrow(priors)],num_states,num_states)
  solver = ValueIterationSolver(max_iterations=1000, belres=1e-6, include_Q=true)#, verbose=true) # initializes the Solver type
  vip = solve(solver, mdp)
  #li = LinearIndices((num_chains, num_states))
  new_policy = FunctionPolicy(curry(curry(curry(curry(curry(ucb_pol, 1), vip), priors),i), actions))
  if start_state == nothing
    return Iterators.Stateful(stepthrough(deepcopy(true_mdp), new_policy, "s,a,r,sp,t",
                              max_steps=steps))
  else
     return Iterators.Stateful(stepthrough(deepcopy(true_mdp), new_policy, start_state, "s,a,r,sp,t",
                              max_steps=steps))
  end
end

ssgn_mdp_builder (generic function with 1 method)

In [15]:
CartesianIndices(size(m.Rs))[1]

CartesianIndex(1, 1)

In [16]:

using DataFrames
using Statistics
function ucb_update_priors(priors, hist, true_vals, true_mdp)
    # each state must be visted me times to update the prior
    # me is # of S*D^2 s= states, D =diameter (https://www.aaai.org/ocs/index.php/AAAI/AAAI15/paper/view/9542/9919)
    #     (D defined in http://www.jmlr.org/papers/volume11/jaksch10a/jaksch10a.pdf)
    ## ^^^ Actually, think should just do this statistically, and if n > 2, then we can adjust the prior.
    println(hist)
    #Any[(1, 1, 1, 1, 1.79837, 2), (1, 1, 2, 2, 0, 1), (1, 1, 3, 1, 2.04313, 2), (1, 1, 4, 2, 0, 1), (1, 1, 5, 1, -0.693505, 2)]
    df = DataFrame(epoch = [x[1] for x in hist], agent=[x[2] for x in hist],
               time = [x[3] for x in hist], state = [x[4] for x in hist], state2 = [x[5] for x in hist],
        reward = [x[6] for x in hist])
    new_theta = []
    new_std = []
    new_c = []
    cs = CartesianIndices(size(true_mdp.Rs))
    for arow in eachrow(priors)
        if nrow(df[(df.state.==cs[arow.c][1]).&(df.state2.==cs[arow.c][1]),:]) > 1
            mu_p = arow.theta
            true_std = true_vals[true_vals.c.==arow.c,:].std[1]
            s_p_2 = arow.std^2
            mu_s = mean(df[(df.state.==cs[arow.c][1]).&(df.state2.==cs[arow.c][1]),:].reward)
            s_s_2 = var(df[(df.state.==cs[arow.c][1]).&(df.state2.==cs[arow.c][1]),:].reward)
            cnt = nrow(df[(df.state.==cs[arow.c][1]).&(df.state2.==cs[arow.c][1]),:])
            #tmp_theta =  (s_p_2/(s_s_2 + s_p_2)) * mu_s + (s_s_2/(s_s_2 + s_p_2)) * mu_p
            #tmp_std = 1/(1/s_p_2 + 1/s_s_2)
            tmp_theta =  (s_p_2/(true_std/cnt + s_p_2)) * mu_s + (s_s_2/(true_std/cnt + s_p_2)) * mu_p
            tmp_std = 1/(1/s_p_2 + cnt/true_std)
        else
            tmp_theta =  arow.theta
            tmp_std = arow.std
        end
        push!(new_c, arow.c)
        push!(new_theta, tmp_theta)
        push!(new_std, tmp_std)
    end
    new_priors = DataFrame(theta=new_theta, std=new_std, c=new_c)
    println("posterior: $new_priors")
    return new_priors
end

ucb_update_priors (generic function with 1 method)

In [17]:
using Distributions
function do_runs(nruns, epochs, num_agents,  num_states, H, update_priors_check, update_priors,
                 mdp_iter_builder, is_thompson_sampling, base, rng)
    # UCB

    # setup constants here

    # So best idea:
    # for ucb:
    # we generate an mdp with mu + sigma rewards (priors) for each end node.
    #  we collect history until the criteria in PAC-EXPLORE then update the prior
    #
    # for thompson sampling
    # at each time step (might skip this for this problem and save for max rew path)
    #   take rewards and use to create a posterior.
    # for seed sampling
    # at beginning of episode
    #   each agent generates a new random "seed"
    #   at each time step
    #     each agent generates a new mdp based on a deterministic mapping from seed to rewards (which also takes
    #              history into account)
    # this needs to update the actual reward.

    ## PC notes
    ## theta_c ~ N(0, 100 + c)
    r_mean = 0
    r_std = 1
    ## sigma is 1
    ## Reward is noisy ~ N(theta_c, sigma^2)
    ## Initial prior mean is N(theta_c, sigma^2 + c)
    runs = []
    true_values_list = DataFrame( run = [],
                                 c = [],
                                 theta = [],
                                 std = [])
    for run in 1:nruns
        println("num_states: $num_states")
        mdp = build_random_path(num_states)
        th_c = reshape(mdp.Rs, 1,:)
        #th_c[1] = 300
        print("th_c:::::")
        println(th_c)
        th_c_sd = sqrt(var(th_c))
        r_std = th_c_sd
        num_rewards = length(th_c)
        th_c = dropdims(th_c;dims=1)
        state_pairs = [(i,j) for i in keys(mdp.trans) for j in mdp.trans[i] ]
        println(state_pairs)
        true_values = DataFrame( run = [run for i in 1:num_rewards],
                                 c = [i for i in 1:num_rewards],
                                 theta = th_c,
                                 std = [r_std  for i in 1:num_rewards])
        println("true mdp: $true_values")
        #priors = DataFrame(theta = [th_c[i] for i in 1:num_chains],
        priors = DataFrame(theta = [0 for i in 1:num_rewards],
                           std = [(sqrt(r_std^2) + i) for i in 1:num_rewards],
                           c = [i for i in 1:num_rewards])
        println("priors: $priors")
        do_update_priors() =  false
        # priors = randn(rng, Float32, num_chains) .* (r_std + c) .+ r_mean
        hist = run_chain!(
                   mdp_iter_builder=mdp_iter_builder,
                   true_mdp=mdp,
                   do_update_priors=update_priors_check,
                   update_priors=update_priors,
                   priors=priors,
                   true_vals=true_values,
                   n_agents=num_agents,
                   num_states=num_states,
                   num_chains=1,
                   epochs=epochs,
                   steps=H,
                   is_thompson_sampling=is_thompson_sampling,
                   rng=rng) #,
                   #rev_action_map=rev_action_map)
        #print(Q_tables)
        println("(e,i,t,st,r)")
        hist
        for (e, ag, t, st, sp, r)  in hist
          push!(runs, (run, e, ag, t, st, sp, r))
        end
        append!(true_values_list, true_values)

    end
    return (true_values_list, runs)
end


do_runs (generic function with 1 method)

In [18]:
do_update_priors() =  false 
rng = MersenneTwister()
base=100
#do_runs(nruns, epochs, num_agents,  num_states, H, update_priors_check, update_priors,
#                 mdp_iter_builder, is_thompson_sampling, base, rng)
true_values_list, runs = do_runs(1, 1, 2,  2, 10,
                                                 do_update_priors, ucb_update_priors, ucb_mdp_builder, false,
                                                 base, rng)

num_states: 2
p:0.6931471805599453p:0.6931471805599453p:0.6931471805599453p:0.6931471805599453Dict{Int64,Array{Int64,N} where N}(2=>[1, 2],1=>[1, 2])th_c:::::[2.0 3.0 4.0 6.0]
Tuple{Int64,Int64}[(2, 1), (2, 2), (1, 1), (1, 2)]
true mdp: 4×4 DataFrame
│ Row │ run   │ c     │ theta   │ std     │
│     │ Int64 │ Int64 │ Float64 │ Float64 │
├─────┼───────┼───────┼─────────┼─────────┤
│ 1   │ 1     │ 1     │ 2.0     │ 1.70783 │
│ 2   │ 1     │ 2     │ 3.0     │ 1.70783 │
│ 3   │ 1     │ 3     │ 4.0     │ 1.70783 │
│ 4   │ 1     │ 4     │ 6.0     │ 1.70783 │
priors: 4×3 DataFrame
│ Row │ theta │ std     │ c     │
│     │ Int64 │ Float64 │ Int64 │
├─────┼───────┼─────────┼───────┤
│ 1   │ 0     │ 2.70783 │ 1     │
│ 2   │ 0     │ 3.70783 │ 2     │
│ 3   │ 0     │ 4.70783 │ 3     │
│ 4   │ 0     │ 5.70783 │ 4     │
e: 1, t: 1, agent 1, result: (s = [1, 1], a = 2, r = 4.0, sp = [1, 2], t = 1)
e: 1, t: 2, agent 1, result: (s = [1, 2], a = 2, r = 6.0, sp = [1, 2], t = 2)
e: 1, t: 3, agent 1, resu

(4×4 DataFrame
│ Row │ run │ c   │ theta │ std     │
│     │ Any │ Any │ Any   │ Any     │
├─────┼─────┼─────┼───────┼─────────┤
│ 1   │ 1   │ 1   │ 2.0   │ 1.70783 │
│ 2   │ 1   │ 2   │ 3.0   │ 1.70783 │
│ 3   │ 1   │ 3   │ 4.0   │ 1.70783 │
│ 4   │ 1   │ 4   │ 6.0   │ 1.70783 │, Any[(1, 1, 1, 1, 1, 2, 1.1766), (1, 1, 1, 2, 2, 2, 3.77678), (1, 1, 2, 1, 1, 2, 0.882483), (1, 1, 1, 3, 2, 2, 3.03993), (1, 1, 2, 2, 2, 2, 3.4433), (1, 1, 1, 4, 2, 2, 2.3605), (1, 1, 2, 3, 2, 2, 4.97251), (1, 1, 1, 5, 2, 2, 2.83886), (1, 1, 2, 4, 2, 2, 2.96507), (1, 1, 1, 6, 2, 2, 3.18522), (1, 1, 2, 5, 2, 2, 3.60071), (1, 1, 1, 7, 2, 2, 2.91703), (1, 1, 2, 6, 2, 2, 1.70613), (1, 1, 1, 8, 2, 2, 3.07544), (1, 1, 2, 7, 2, 2, 3.17513), (1, 1, 1, 9, 2, 2, 1.86717), (1, 1, 2, 8, 2, 2, 2.25575), (1, 1, 1, 10, 2, 2, 3.25299), (1, 1, 2, 9, 2, 2, 3.22348), (1, 1, 2, 10, 2, 2, 2.50098)])

ssgn_mdp_builder (generic function with 1 method)

In [20]:

function get_average_regret(results,  theta, num_states)
    # 1, 1, 1, 1, 1, 2
    println("results: $results")
    println("theta: $theta")
    
    li = LinearIndices((num_states, num_states))
    df = DataFrame(run = [x[1] for x in results], epoch = [x[2] for x in results], agent=[x[3] for x in results],
               time = [x[4] for x in results], c = [li[x[5],x[6]] for x in results],
               reward = [x[7] for x in results])
    df_R = df[df.reward.!==0,:]
    th_max = by(theta, :run, :theta => maximum)
    th_min = by(theta, :run, :theta => minimum)
    theta_joined = join(join(theta, th_max, on = :run), th_min, on = :run)
    df_joined = join(df_R, theta_joined, on = [:run,:c] )
    # needs to be max theta
    df_joined.Regret = df_joined.theta_maximum - df_joined.reward
    df_joined
end


get_average_regret (generic function with 1 method)

In [39]:

# Full loop
function run_all(rng)
    full_df = nothing;
    base = 100

    for num_agents in [1,10, 100]
        println("*****************number of agents:$num_agents*******************")
        num_chains = 10
        num_states = 5
        epochs = 1
        nruns = 2
        H = 12
        do_update_priors() = true
        for run_type in ["UCB"]#"Thompson Sampling", "Seed Sampling", "UCB"]
            println("------------------Run Type:$run_type------------------------")
            if run_type == "Thompson Sampling"
                true_values_list, runs = do_runs(nruns, epochs, num_agents, num_states, H,
                                                 do_update_priors, ucb_update_priors,  ssgn_mdp_builder, true,
                                                 base, rng)
            elseif run_type == "UCB"
                true_values_list, runs = do_runs(nruns, epochs, num_agents, num_states, H,
                                                 do_update_priors, ucb_update_priors, ucb_mdp_builder, false,
                                                 base, rng)
            else
                true_values_list, runs = do_runs(nruns, epochs, num_agents, num_states, H,
                                                 do_update_priors, ucb_update_priors,  ssgn_mdp_builder, false,
                                                 base, rng)

            end

            chain_len = num_states
            num_chains = num_chains
            regret = get_average_regret(runs, true_values_list, num_states)
            regret.name = [run_type for i in 1:nrow(regret)]
            regret.num_agents = [num_agents for i in 1:nrow(regret)]
            regret.base = [base for i in 1:nrow(regret)]
            if full_df == nothing
                full_df = regret
            else
                append!(full_df,regret)
            end
        end
    end

    println("***********************Done!!!*********************************")
    full_df[full_df[:Regret] .< 0.0,:Regret] = 0.0
    CSV.write("mr_simulation_1.csv", full_df)
    full_df
end

rng = MersenneTwister()


MersenneTwister(UInt32[0x2b1314b2, 0x7bff17ea, 0x4ede4767, 0xdba48fd3], Random.DSFMT.DSFMT_state(Int32[1996117774, 1073183272, -866805005, 1072862113, -865093854, 1072905809, -148966446, 1073413079, -1825174978, 1073141687  …  -1397209865, 1073265543, 163202684, 1073459915, -549559453, 1954507132, 2095516093, -1590902069, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000

In [36]:
run_all(rng)

*****************number of agents:1*******************
------------------Run Type:UCB------------------------
num_states: 5
p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401p:0.6437751649736401Dict{Int64,Array{Int64,N} where N}(4=>[1, 2, 4, 5],2=>[1, 2, 5],3=>[2, 5],5=>[2, 3, 4, 5],1=>[1, 2])th_c:::::[5.0 3.0 0.0 0.0 2.0 10.0 6.0 8.0 12.0 4.0 0.0 0.0 0.0 18.0 0.0 20.0 0.0 0.0 24.0 0.0 25.0 15.0 20.0 30.0 0.0]
Tuple{Int64,Int64}[(4, 1), (4, 2), (4, 4), (4, 5), (2, 1), (2, 2), (2, 5), (3, 2), (3, 5), (5, 2), (5, 3), (5, 4), (5, 5), (1, 1), (1, 2)]
true mdp: 25×4 Da

e: 1, t: 1, agent 1, result: (s = [1, 1], a = 5, r = 0.0, sp = [1, 5], t = 1)
e: 1, t: 2, agent 1, result: (s = [1, 5], a = 5, r = 0.0, sp = [1, 5], t = 2)
e: 1, t: 3, agent 1, result: (s = [1, 5], a = 5, r = 0.0, sp = [1, 5], t = 3)
e: 1, t: 4, agent 1, result: (s = [1, 5], a = 5, r = 0.0, sp = [1, 5], t = 4)
e: 1, t: 5, agent 1, result: (s = [1, 5], a = 5, r = 0.0, sp = [1, 5], t = 5)
e: 1, t: 6, agent 1, result: (s = [1, 5], a = 5, r = 0.0, sp = [1, 5], t = 6)
e: 1, t: 7, agent 1, result: (s = [1, 5], a = 5, r = 0.0, sp = [1, 5], t = 7)
e: 1, t: 8, agent 1, result: (s = [1, 5], a = 5, r = 0.0, sp = [1, 5], t = 8)
e: 1, t: 9, agent 1, result: (s = [1, 5], a = 5, r = 0.0, sp = [1, 5], t = 9)
e: 1, t: 10, agent 1, result: (s = [1, 5], a = 5, r = 0.0, sp = [1, 5], t = 10)
e: 1, t: 11, agent 1, result: (s = [1, 5], a = 5, r = 0.0, sp = [1, 5], t = 11)
e: 1, t: 12, agent 1, result: (s = [1, 5], a = 5, r = 0.0, sp = [1, 5], t = 12)
Any[(1, 1, 1, 1, 5, 7.09989), (1, 1, 2, 5, 5, 2.09638), (1

e: 1, t: 10, agent 2, result: (s = [1, 5], a = 5, r = 10.0, sp = [1, 5], t = 10)
e: 1, t: 10, agent 3, result: (s = [1, 5], a = 5, r = 10.0, sp = [1, 5], t = 10)
e: 1, t: 9, agent 4, result: (s = [1, 5], a = 5, r = 10.0, sp = [1, 5], t = 9)
e: 1, t: 9, agent 5, result: (s = [1, 5], a = 5, r = 10.0, sp = [1, 5], t = 9)
e: 1, t: 6, agent 6, result: (s = [1, 5], a = 5, r = 10.0, sp = [1, 5], t = 6)
e: 1, t: 5, agent 7, result: (s = [1, 5], a = 5, r = 10.0, sp = [1, 5], t = 5)
e: 1, t: 5, agent 8, result: (s = [1, 5], a = 5, r = 10.0, sp = [1, 5], t = 5)
e: 1, t: 4, agent 9, result: (s = [1, 5], a = 5, r = 10.0, sp = [1, 5], t = 4)
e: 1, t: 1, agent 10, result: (s = [1, 1], a = 5, r = 25.0, sp = [1, 5], t = 1)
e: 1, t: 11, agent 2, result: (s = [1, 5], a = 5, r = 10.0, sp = [1, 5], t = 11)
e: 1, t: 11, agent 3, result: (s = [1, 5], a = 5, r = 10.0, sp = [1, 5], t = 11)
e: 1, t: 10, agent 4, result: (s = [1, 5], a = 5, r = 10.0, sp = [1, 5], t = 10)
e: 1, t: 10, agent 5, result: (s = [1, 5]

e: 1, t: 11, agent 7, result: (s = [1, 5], a = 5, r = 10.0, sp = [1, 5], t = 11)
e: 1, t: 11, agent 8, result: (s = [1, 5], a = 5, r = 10.0, sp = [1, 5], t = 11)
e: 1, t: 10, agent 9, result: (s = [1, 5], a = 5, r = 10.0, sp = [1, 5], t = 10)
e: 1, t: 7, agent 10, result: (s = [1, 5], a = 5, r = 10.0, sp = [1, 5], t = 7)
e: 1, t: 12, agent 7, result: (s = [1, 5], a = 5, r = 10.0, sp = [1, 5], t = 12)
Any[(1, 1, 1, 1, 5, 6.77382), (1, 1, 2, 5, 5, 0), (1, 1, 3, 5, 5, 0), (1, 2, 1, 1, 5, 4.58127), (1, 3, 1, 1, 5, 4.7541), (1, 1, 4, 5, 5, 0), (1, 2, 2, 5, 5, 0), (1, 3, 2, 5, 5, 0), (1, 4, 1, 1, 5, 5.49545), (1, 5, 1, 1, 5, 4.19078), (1, 1, 5, 5, 5, 0), (1, 2, 3, 5, 5, 0), (1, 3, 3, 5, 5, 0), (1, 4, 2, 5, 5, 0), (1, 5, 2, 5, 5, 0), (1, 1, 6, 5, 5, 0), (1, 2, 4, 5, 5, 0), (1, 3, 4, 5, 5, 0), (1, 4, 3, 5, 5, 0), (1, 5, 3, 5, 5, 0), (1, 1, 7, 5, 5, 0), (1, 2, 5, 5, 5, 0), (1, 3, 5, 5, 5, 0), (1, 4, 4, 5, 5, 0), (1, 5, 4, 5, 5, 0), (1, 6, 1, 1, 5, 4.91363), (1, 1, 8, 5, 5, 0), (1, 2, 6, 5, 5, 0

Any[(1, 1, 1, 1, 5, 6.77382), (1, 1, 2, 5, 5, 0), (1, 1, 3, 5, 5, 0), (1, 2, 1, 1, 5, 4.58127), (1, 3, 1, 1, 5, 4.7541), (1, 1, 4, 5, 5, 0), (1, 2, 2, 5, 5, 0), (1, 3, 2, 5, 5, 0), (1, 4, 1, 1, 5, 5.49545), (1, 5, 1, 1, 5, 4.19078), (1, 1, 5, 5, 5, 0), (1, 2, 3, 5, 5, 0), (1, 3, 3, 5, 5, 0), (1, 4, 2, 5, 5, 0), (1, 5, 2, 5, 5, 0), (1, 1, 6, 5, 5, 0), (1, 2, 4, 5, 5, 0), (1, 3, 4, 5, 5, 0), (1, 4, 3, 5, 5, 0), (1, 5, 3, 5, 5, 0), (1, 1, 7, 5, 5, 0), (1, 2, 5, 5, 5, 0), (1, 3, 5, 5, 5, 0), (1, 4, 4, 5, 5, 0), (1, 5, 4, 5, 5, 0), (1, 6, 1, 1, 5, 4.91363), (1, 1, 8, 5, 5, 0), (1, 2, 6, 5, 5, 0), (1, 3, 6, 5, 5, 0), (1, 4, 5, 5, 5, 0), (1, 5, 5, 5, 5, 0), (1, 6, 2, 5, 5, 0), (1, 7, 1, 1, 5, 6.45965), (1, 8, 1, 1, 5, 4.7373), (1, 1, 9, 5, 5, 0), (1, 2, 7, 5, 5, 0), (1, 3, 7, 5, 5, 0), (1, 4, 6, 5, 5, 0), (1, 5, 6, 5, 5, 0), (1, 6, 3, 5, 5, 0), (1, 7, 2, 5, 5, 0), (1, 8, 2, 5, 5, 0), (1, 9, 1, 1, 5, 5.19413), (1, 1, 10, 5, 5, 0), (1, 2, 8, 5, 5, 0), (1, 3, 8, 5, 5, 0), (1, 4, 7, 5, 5, 0), (1,

...

Any[(1, 1, 1, 1, 5, 4.21456), (1, 1, 2, 5, 5, 0), (1, 2, 1, 1, 5, 3.7766), (1, 3, 1, 1, 5, 5.9944), (1, 1, 3, 5, 5, 0), (1, 2, 2, 5, 5, 0), (1, 3, 2, 5, 5, 0), (1, 1, 4, 5, 5, 0), (1, 2, 3, 5, 5, 0), (1, 3, 3, 5, 5, 0), (1, 1, 5, 5, 5, 0), (1, 2, 4, 5, 5, 0), (1, 3, 4, 5, 5, 0), (1, 4, 1, 1, 5, 3.12838), (1, 1, 6, 5, 5, 0), (1, 2, 5, 5, 5, 0), (1, 3, 5, 5, 5, 0), (1, 4, 2, 5, 5, 0), (1, 1, 7, 5, 5, 0), (1, 2, 6, 5, 5, 0), (1, 3, 6, 5, 5, 0), (1, 4, 3, 5, 5, 0), (1, 1, 8, 5, 5, 0), (1, 2, 7, 5, 5, 0), (1, 3, 7, 5, 5, 0), (1, 4, 4, 5, 5, 0), (1, 1, 9, 5, 5, 0), (1, 2, 8, 5, 5, 0), (1, 3, 8, 5, 5, 0), (1, 4, 5, 5, 5, 0), (1, 5, 1, 1, 5, 5.54869), (1, 6, 1, 1, 5, 4.63863), (1, 1, 10, 5, 5, 0), (1, 2, 9, 5, 5, 0), (1, 3, 9, 5, 5, 0), (1, 4, 6, 5, 5, 0), (1, 5, 2, 5, 5, 0), (1, 6, 2, 5, 5, 0), (1, 1, 11, 5, 5, 0), (1, 2, 10, 5, 5, 0), (1, 3, 10, 5, 5, 0), (1, 4, 7, 5, 5, 0), (1, 5, 3, 5, 5, 0), (1, 6, 3, 5, 5, 0), (1, 7, 1, 1, 5, 4.70141), (1, 8, 1, 1, 5, 5.07716), (1, 9, 1, 1, 5, 6.44914), 

 28, 10, 5, 2, 0), (1, 29, 8, 5, 2, 0), (1, 30, 5, 2, 5, 5.0178), (1, 31, 5, 2, 5, 3.26063), (1, 32, 4, 5, 2, 0), (1, 33, 2, 5, 2, 0), (1, 34, 2, 5, 2, 0), (1, 35, 2, 5, 2, 0), (1, 36, 1, 1, 5, 4.85409), (1, 27, 12, 5, 2, 0), (1, 28, 11, 2, 5, 2.61986), (1, 29, 9, 2, 5, 2.5842), (1, 30, 6, 5, 2, 0), (1, 31, 6, 5, 2, 0), (1, 32, 5, 2, 5, 3.17415), (1, 33, 3, 2, 5, 2.21591), (1, 34, 3, 2, 5, 1.47066), (1, 35, 3, 2, 5, 1.10256), (1, 36, 2, 5, 2, 0), (1, 37, 1, 1, 5, 5.15004), (1, 38, 1, 1, 5, 4.21114), (1, 39, 1, 1, 5, 6.20187), (1, 28, 12, 5, 2, 0), (1, 29, 10, 5, 2, 0), (1, 30, 7, 2, 5, 2.73544), (1, 31, 7, 2, 5, 3.3844), (1, 32, 6, 5, 2, 0), (1, 33, 4, 5, 2, 0), (1, 34, 4, 5, 2, 0), (1, 35, 4, 5, 2, 0), (1, 36, 3, 2, 5, 4.00667), (1, 37, 2, 5, 2, 0), (1, 38, 2, 5, 2, 0), (1, 39, 2, 5, 2, 0), (1, 40, 1, 1, 5, 6.3199), (1, 29, 11, 2, 5, 3.61618), (1, 30, 8, 5, 2, 0), (1, 31, 8, 5, 2, 0), (1, 32, 7, 2, 5, 3.7142), (1, 33, 5, 2, 5, 1.75885), (1, 34, 5, 2, 5, 1.72209), (1, 35, 5, 2, 5, 4.48

Excessive output truncated after 524289 bytes.

, 1, 5, 4.70299), (1, 19, 12, 5, 2, 0), (1, 20, 12, 5, 2, 0), (1, 21, 12, 5, 2, 0), (1, 22, 9, 2, 5, 3.46689), (1, 23, 8, 5, 2, 0), (1, 24, 6, 5, 2, 0), (1, 25, 4, 5, 2, 0), (1, 26, 4, 5, 2, 0), (1, 27, 3, 2, 5, 2.86174), (1, 28, 2, 5, 2, 0), (1, 22, 10, 5, 2, 0), (1, 23, 9, 2, 5, 2.81352), (1, 24, 7, 2, 5, 3.24196), (1, 25, 5, 2, 5, 2.79619), (1, 26, 5, 2, 5, 4.70917), (1, 27, 4, 5, 2, 0), (1, 28, 3, 2, 5, 3.18938), (1, 29, 1, 1, 5, 5.9951), (1, 22, 11, 2, 5, 4.87338), (1, 23, 10, 5, 2, 0), (1, 24, 8, 5, 2, 0), (1, 25, 6, 5, 2, 0), (1, 26, 6, 5, 2, 0), (1, 27, 5, 2, 5, 2.94268), (1, 28, 4, 5, 2, 0), (1, 29, 2, 5, 2, 0), (1, 22, 12, 5, 2, 0), (1, 23, 11, 2, 5, 1.77672), (1, 24, 9, 2, 5, 3.67352), (1, 25, 7, 2, 5, 2.35225), (1, 26, 7, 2, 5, 4.02516), (1, 27, 6, 5, 2, 0), (1, 28, 5, 2, 5, 2.27408), (1, 29, 3, 2, 5, 3.55222), (1, 23, 12, 5, 2, 0), (1, 24, 10, 5, 2, 0), (1, 25, 8, 5, 2, 0), (1, 26, 8, 5, 2, 0), (1, 27, 7, 2, 5, 2.82425), (1, 28, 6, 5, 2, 0), (1, 29, 4, 5, 2, 0), (1, 30, 1,

,run,epoch,agent,time,c,reward,theta,std,theta_maximum,theta_minimum,Regret,name,num_agents,base
,Int64,Int64,Int64,Int64,Int64,Float64,Any,Any,Float64,Float64,Float64,String,Int64,Int64
1,1,1,1,1,21,5.64523,25.0,9.60868,30.0,0.0,24.3548,UCB,1,100
2,1,1,1,2,25,2.20823,0.0,9.60868,30.0,0.0,27.7918,UCB,1,100
3,1,1,1,3,25,1.91779,0.0,9.60868,30.0,0.0,28.0822,UCB,1,100
4,1,1,1,4,25,2.75342,0.0,9.60868,30.0,0.0,27.2466,UCB,1,100
5,1,1,1,5,25,1.39866,0.0,9.60868,30.0,0.0,28.6013,UCB,1,100
6,1,1,1,6,25,2.26281,0.0,9.60868,30.0,0.0,27.7372,UCB,1,100
7,1,1,1,7,25,1.6441,0.0,9.60868,30.0,0.0,28.3559,UCB,1,100
8,1,1,1,8,25,1.67981,0.0,9.60868,30.0,0.0,28.3202,UCB,1,100
9,1,1,1,9,25,0.855574,0.0,9.60868,30.0,0.0,29.1444,UCB,1,100
